In [78]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import numpy as np
import re
import plotly_express as px
from dash.dependencies import Input, Output
rootfold = '/home/j0hndoe/Documents/git/reddit-disinformation'

In [2]:
def get_flag_perc(df, groups):
    return df.\
        groupby(groups)\
            [['n_subm_kept','ISFLAG']].\
                agg(sum).sort_index().reset_index().\
        assign(perc_FLAG = lambda df: df.ISFLAG/df.n_subm_kept)

In [205]:
df_flags = pd.read_csv(rootfold+"/dashboard/data/app_flags_day.csv")
df_subm = pd.read_csv(rootfold+"/dashboard/data/app_subr_week.csv")

df_subm['week'] = df_subm['week'].astype('int')
min_week, max_week = df_subm['week'].min(), df_subm['week'].max()
weekrange = [min_week, max_week]
weekrange = [15, 29]

filtered_df_subm = df_day[(df_subm['week'].between(*weekrange))]# & (df['varr']==varr)]
filtered_df_subm['ISFLAG'] = filtered_df_subm.flag.copy()#loc[:,flagtypes].sum(axis=1)>0
### Aggregate
filtered_df_week = get_flag_perc(
    filtered_df_subm,
    groups = ['subreddit_cat','subreddit','week'])
filtered_df_agg = get_flag_perc(
    filtered_df_subm,
    groups = ['subreddit_cat','subreddit'])

<ipython-input-205-30c6cbb27fd9>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [206]:
from datetime import datetime
x = "2020-01-05"
def revdate(datestring, year = False):
    if year:
        return datetime.strptime(datestring, "%Y-%m-%d").strftime("%m %d, %Y")
    else:
        return datetime.strptime(datestring, "%Y-%m-%d").strftime("%m %d")
datetime.strptime(x, "%Y-%m-%d").strftime("%b %d, %Y")
revdate(x, year=True)

'01 05, 2020'

In [207]:
week_day_df = df_flags.groupby('week')['day'].agg(['min','max'])
week_day_df['label'] = [r[1]['min'][8:10] + '/' + r[1]['min'][5:7] + ' - ' + r[1]['max'][8:10] + '/' + r[1]['max'][5:7]
                        for r in week_day_df.iterrows()]
week_day_df['label'] = [r[1]['min'][5:7] + '/\n' + r[1]['min'][8:10]
                        for r in week_day_df.iterrows()]
week_day_df

,min,max,label
week,,,
1.0,2020-01-01,2020-01-05,01/\n01
2.0,2020-01-06,2020-01-12,01/\n06
3.0,2020-01-13,2020-01-19,01/\n13
4.0,2020-01-20,2020-01-26,01/\n20
5.0,2020-01-27,2020-02-02,01/\n27
6.0,2020-02-03,2020-02-09,02/\n03
7.0,2020-02-10,2020-02-16,02/\n10
8.0,2020-02-17,2020-02-23,02/\n17
9.0,2020-02-24,2020-03-01,02/\n24


In [211]:
filtered_df_week_cat = get_flag_perc(filtered_df_week, ['subreddit_cat','week'])
tickvals0 = list(range(filtered_df_week_cat.week.min(),
           filtered_df_week_cat.week.max(),
           1)) + [filtered_df_week_cat.week.max()]
tickvals0 = subr_line_all.data[0].x
ticklabs0 = week_day_df['label'][tickvals0]
subr_line_all = px.line(
                    filtered_df_week_cat,
                    x = "week",
                    y = "ISFLAG",
                    color = "subreddit_cat")

subr_line_all.show()

In [300]:
rng0 = [1,10]
def get_xlabs(xrange, nticks =5):    
    rng = xrange[1] - xrange[0]
    ticklst = list(range(xrange[0], xrange[1], rng // nticks+1)) 
    return ticklst + [xrange[1]]
    
get_xlabs(rng0)

[1, 3, 5, 7, 9, 10]

In [309]:
min([min(a.x) for a in subr_line_all.data])

15

In [243]:

subr_line_all.update_layout(xaxis = dict(tickmode = 'array',
                                         tickvals = tickvals0,
                                         ticktext = ticklabs0
                                         ))

In [184]:
dir(subr_line_all)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_annotation_like',
 '_allow_disable_validation',
 '_animation_duration_validator',
 '_animation_easing_validator',
 '_batch_layout_edits',
 '_batch_trace_edits',
 '_bracket_re',
 '_build_dispatch_plan',
 '_build_update_params_from_batch',
 '_config',
 '_data',
 '_data_defaults',
 '_data_objs',
 '_data_validator',
 '_dispatch_layout_change_callbacks',
 '_dispatch_trace_change_callbacks',
 '_frame_objs',
 '_frames_validator',
 '_get_child_prop_defaults',
 '_get_child_props',
 '_grid_ref',
 '_grid_str',
 '_in_batch_mode',
 '_index_is',
 '_init_child_props',
 

In [188]:
subr_line_all.data[0].x

array([25, 26, 27, 28, 29])

In [117]:
fig_subr_treemap = px.treemap(filtered_df_agg, 
                      path=['subreddit_cat', 'subreddit'], 
                      values='ISFLAG',
                      color='subreddit_cat')
fig_subr_treemap.show()

### Cluster analysis

In [483]:
subm = pd.read_csv(rootfold+"/dashboard/data/app_subm_day.csv")

In [484]:
from sklearn.cluster import MiniBatchKMeans
flagtypes = list(subm.loc[:,"disinformation":"other"].columns)
nclust = 5

In [485]:
filtered = subm.copy()        
filtered = filtered[(filtered['week'].between(*weekrange))]
filtered['ISFLAG'] = filtered[flagtypes].copy().sum(axis=1)
filtered = filtered[filtered['ISFLAG']>0]
filtered = filtered[filtered['subm_covid']]
### Run cluster analysis
if nclust>1:
    X = filtered[['PC01', 'PC02', 'PC03', 'PC04', 'PC05']]
    cluster = MiniBatchKMeans(n_clusters=nclust, batch_size=1000)
    cluster.fit(X)
    filtered['cluster'] = (cluster.predict(X)+1).astype(str)
#filtered

In [486]:
cluster.cluster_centers_

array([[-0.00731287, -0.0025561 , -0.00695286,  0.00151926,  0.00328427],
       [-0.00409679,  0.00479816,  0.00129368,  0.00529476,  0.00299267],
       [-0.00486317,  0.00612195, -0.00176477, -0.00939114, -0.00149239],
       [-0.00448208,  0.00122206,  0.00416373,  0.00065331, -0.00187014],
       [-0.00419501,  0.0060616 , -0.00276441,  0.00337155, -0.00344449]])

In [487]:
np.inner(X,cluster.cluster_centers_[0,:])

array([ 3.63513357e-05,  2.37056720e-05,  4.08405430e-05, ...,
        5.56973112e-05,  4.50911670e-05, -1.73725544e-05])

In [488]:
np.inner(X,cluster.cluster_centers_[0,:])[5022]

-1.6177169136743895e-05

In [501]:
filtered[['subm_title','subreddit']].iloc[sel_submissions_index]

,subm_title,subreddit
8998,Trump says he's been taking hydroxychloroquine...,Coronavirus
8891,Trump says he's been taking hydroxychloroquine...,politics
8868,Trump says he's been taking hydroxychloroquine...,politics
9037,President Trump says he's been taking hydroxyc...,Conservative
8894,Trump says he's been taking hydroxychloroquine,politics


In [498]:
sel_submissions_index = np.argsort(np.inner(X,cluster.cluster_centers_[0,:]))[-5:]
list(sel_submissions_index)
filtered.subm_title.iloc[sel_submissions_index]

8998    Trump says he's been taking hydroxychloroquine...
8891    Trump says he's been taking hydroxychloroquine...
8868    Trump says he's been taking hydroxychloroquine...
9037    President Trump says he's been taking hydroxyc...
8894       Trump says he's been taking hydroxychloroquine
Name: subm_title, dtype: object

In [377]:
dotprod = X.dot(cluster.cluster_centers_.transpose())
dotprod.columns = ['C' + str(i+1) for i in range(nclust)]
joined = filtered[['subm_title','cluster']].join(dotprod)
for cl in range(nclust):
    print(dotprod.columns[cl])
    print(joined.sort_values(dotprod.columns[cl], ascending=False).subm_title.unique()[:20])

C1
['CDC says U.S. has ‘way too much virus’ to control pandemic as cases surge across country'
 'Coronavirus Has Now Killed More People in US than 2018-2019 Flu Season'
 "CDC: Coronavirus Deaths Will Be 'Much Much Much Lower' than Projected"
 'New report says coronavirus pandemic could last for two years – and may not subside until 70% of the population has immunity'
 'NDR Study Shows That Many Who Died From Coronavirus Would Have Lived For A Long Time'
 'COVID-19 kills 20 times more people per week on average than flu, scientists estimate'
 'COVID-19 Kills 20 times More People per Week on Average than Flu, Scientists Estimate'
 'COVID-19 at least 10 times deadlier than typical flu, analysis finds'
 'COVID-19 is close to losing its epidemic status in the U.S., according to the CDC'
 'The CDC Released New Death Rate Estimates For The Coronavirus. Many Scientists Say They’re Too Low..'
 'People with coronavirus are dying 10 years earlier than they would have naturally: Study The study di

In [379]:
joined.head()

,subm_title,cluster,C1,C2,C3,C4,C5
0,Essays: How Hackers and Spies Could Sabotage t...,2,0.000016,0.000024,0.000034,0.000089,0.000018
1,76% of Americans have gained up to 16 pounds d...,1,0.000097,0.000045,0.000025,-0.000006,0.000029
2,Making Masks Fun! Tips for Getting Kids to Emb...,1,0.000113,0.000018,0.000042,-0.000019,0.000070
3,"Florida Breaks Record, Over 15,000 COVID-19 Ca...",5,0.000087,0.000033,0.000008,-0.000032,0.000085
4,Premier Doug Ford reveals Stage 3 of Ontario's...,5,0.000034,0.000021,0.000016,-0.000032,0.000046


In [445]:
[{'label': 'Cluster '+str(i), 'value': i} for i in range(1,nclust+1)]

[{'label': 'Cluster 1', 'value': 1},
 {'label': 'Cluster 2', 'value': 2},
 {'label': 'Cluster 3', 'value': 3},
 {'label': 'Cluster 4', 'value': 4},
 {'label': 'Cluster 5', 'value': 5}]

#### TF-IDF

In [380]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [392]:
tfidfer = TfidfVectorizer(min_df=5, stop_words='english')
tf_idf_matrix = tfidfer.fit_transform(joined.subm_title)
feature_names = np.array(tfidfer.get_feature_names())
tf_idf_matrix

<8071x2710 sparse matrix of type '<class 'numpy.float64'>'
	with 66591 stored elements in Compressed Sparse Row format>

In [397]:
tf_idf_matrix[:10,:10].data

array([0.29242875])

In [400]:
indices = np.argsort(tfidfer.idf_)[::-1]
features = tfidfer.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
print(top_features)

['exclude', 'dept', 'rescue', 'demanding', 'requirement', 'zone', 'surging', 'deliberately', 'surpasses', 'laboratories', 'delhi', 'deleted', 'requests', 'defies', 'deficiency', 'swabs', 'lashes', 'declining', 'declared', 'decades']


In [412]:
def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df
def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25):
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features, top_n)
def top_feats_by_class(Xtr, y, features, min_tfidf=0.1, top_n=25):
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features, ids, min_tfidf=min_tfidf, top_n=top_n)
        feats_df.label = label
        print('\nCLUSTER '+label)
        print(feats_df)
        #return dfs

In [414]:
features = tfidfer.get_feature_names()
top_feats_by_class(tf_idf_matrix, joined.cluster, features, min_tfidf=0.1, top_n=10)


CLUSTER 1
       feature     tfidf
0        covid  0.045350
1           19  0.043838
2  coronavirus  0.038043
3     pandemic  0.035405
4        study  0.026368
5        virus  0.025952
6       deaths  0.025292
7      vaccine  0.023674
8          new  0.022626
9         says  0.021064

CLUSTER 2
       feature     tfidf
0  coronavirus  0.046611
1        covid  0.043238
2           19  0.040035
3          new  0.016862
4       crisis  0.015670
5     lockdown  0.015484
6        world  0.014393
7        death  0.014116
8         says  0.013491
9           uk  0.013095

CLUSTER 3
              feature     tfidf
0               trump  0.172933
1         coronavirus  0.051610
2                says  0.039389
3  hydroxychloroquine  0.024487
4           president  0.024411
5              donald  0.021597
6            pandemic  0.021375
7               covid  0.020948
8            response  0.019860
9               white  0.019593

CLUSTER 4
       feature     tfidf
0        china  0.147709
1   

In [415]:
from wordcloud import WordCloud

In [416]:
wordcloud = WordCloud (
                    background_color = 'white',
                    width = 512,
                    height = 384
                        ).generate(' '.join(joined.subm_title))
plt.imshow(wordcloud) # image show
plt.axis('off') # to off the axis of x and y

NameError: name 'plt' is not defined

In [436]:
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
def cluster_commonwords(texts, nwords = 5):
    allwords = [w for w in ' '.join(texts.str.lower()).split() if w not in STOP_WORDS and re.search('[a-z]',w)]
    return ', '.join([word for word,cnt in Counter(allwords).most_common(nwords)])

In [439]:
c = cluster_commonwords(joined.subm_title, nwords = 10)

In [443]:
STOP_WORDS.union(['coronavirus','covid','covid19','covid-19'])

{"'d",
 "'ll",
 "'m",
 "'re",
 "'s",
 "'ve",
 'a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'both',
 'bottom',
 'but',
 'by',
 'ca',
 'call',
 'can',
 'cannot',
 'coronavirus',
 'could',
 'covid',
 'covid-19',
 'covid19',
 'did',
 'do',
 'does',
 'doing',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'first',
 'five',
 'for',
 'forme

In [446]:
def generate_table(dataframe, max_rows=10):
    return html.Table(
        # Header
        [html.Tr([html.Th(col) for col in dataframe.columns])] +

        # Body
        [html.Tr([
            html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
        ]) for i in range(min(len(dataframe), max_rows))]
    )


In [447]:
generate_table(joined)

Table([Tr([Th('subm_title'), Th('cluster'), Th('C1'), Th('C2'), Th('C3'), Th('C4'), Th('C5')]), Tr([Td('Essays: How Hackers and Spies Could Sabotage the Coronavirus Fight'), Td('2'), Td(1.5990070980698564e-05), Td(2.3945586887275975e-05), Td(3.404477125030537e-05), Td(8.927038272108391e-05), Td(1.7854423987699105e-05)]), Tr([Td('76% of Americans have gained up to 16 pounds during the quarantine, according to Nutrisystem'), Td('1'), Td(9.650327209161662e-05), Td(4.5273559767302794e-05), Td(2.5419302647355936e-05), Td(-6.113591696484178e-06), Td(2.8742864497154713e-05)]), Tr([Td('Making Masks Fun! Tips for Getting Kids to Embrace Face Coverings amid the Coronavirus Pandemic'), Td('1'), Td(0.00011261491152779758), Td(1.7513501464760036e-05), Td(4.177482193240602e-05), Td(-1.886683772317475e-05), Td(7.023248296825236e-05)]), Tr([Td('Florida Breaks Record, Over 15,000 COVID-19 Cases in One Day'), Td('5'), Td(8.715845483356507e-05), Td(3.3471132337147336e-05), Td(7.854037132687516e-06), Td(-